In [1]:
import pyttsx3
import requests
from duckduckgo_search import DDGS
from newspaper import Article
import webbrowser
import time
import wikipedia
from autogen import AssistantAgent

# Initialize text-to-speech
engine = pyttsx3.init()

# Groq API Key (Keep this secret!)
GROQ_API_KEY = "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"

#----------------------------------------------------------------------------------------------------
# Including a new summariser agent
summariser = AssistantAgent(
    name="Summariser",
    llm_config={
        "api_key": "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",  
        "base_url": "https://api.groq.com/openai/v1",
        "model": "llama3-70b-8192"
    }
)
#----------------------------------------------------------------------------------------------------
def ai_giving_response(user_input):
    url = "https://api.groq.com/openai/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3-70b-8192",
        "messages": [{"role": "user", "content": user_input}]
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        response_json = response.json()
        if "choices" in response_json and len(response_json["choices"]) > 0:
            return response_json["choices"][0]["message"]["content"]
        else:
            return "I received an unexpected response."
    else:
        print("Error:", response.text)
        return "I encountered an error."
#----------------------------------------------------------------------------------------------------
def text_to_speech(text):
    engine.say(text)
    engine.runAndWait()
#----------------------------------------------------------------------------------------------------
def play_youtube(query):
    try:
        text_to_speech(f"Searching YouTube for {query}")
        search_query = f"{query} site:youtube.com"
        with DDGS() as ddgs:
            time.sleep(2)
            results = list(ddgs.text(search_query, max_results=1))
            if results:
                url = results[0]['href']
                text_to_speech("Opening YouTube video.")
                print("Opening:", url)
                webbrowser.open_new_tab(url)
            else:
                text_to_speech("No YouTube video found.")
    except Exception as e:
        print("Error:", e)
        text_to_speech("An error occurred while trying to play the video.")
#----------------------------------------------------------------------------------------------------
def get_article_text(url):
    try:
        summary_prompt = f"Give me a clean 5-point summary of the article in this link:\n{url}"

        summary_text = summariser.generate_reply(
            message=summary_prompt,
            sender="user"
        )

        text_to_speech(summary_text)
        return summary_text  # Return the plain string
    except Exception as e:
        print(f"Error fetching summary: {e}")
        return "Sorry, I couldn't summarize the article."

#----------------------------------------------------------------------------------------------------
def is_article_link(url):
    return any(ext in url for ext in [".html", "/article", "/news", "/202"])

def search_and_read(query):
    try:
        with DDGS() as ddgs:
            time.sleep(2)  # Avoid rate limiting
            results = list(ddgs.text(query, max_results=1))
            if results:
                first_result = results[0]
                url = first_result["href"]
                print("Fetching and summarizing:", url)
                if is_article_link(url):
                    content = get_article_text(url)
                    if content:
                        text_to_speech(content[:700])
                    else:
                        text_to_speech("Sorry, I couldn't read the article.")
                else:
                    text_to_speech("That doesn't look like a direct article link. Trying Wikipedia instead.")
                    raise Exception("Invalid link type")
            else:
                raise Exception("No DDG results")

    except Exception as e:
        print(f"Falling back to Wikipedia due to: {e}")
        try:
            summary = wikipedia.summary(query, sentences=3)
            print("Wikipedia Summary:\n", summary)
            text_to_speech("Here's what I found on Wikipedia.")
            text_to_speech(summary)
        except Exception as we:
            print("Wikipedia error:", we)
            text_to_speech("Sorry, I couldn't find anything useful right now.")

#----------------------------------------------------------------------------------------------------

def main():
    password = input("Enter the password to activate assistant: ").lower()
    if password != "jarvis":
        print("Incorrect password. Exiting.")
        return
    
    text_to_speech("Password accepted. How can I assist you?")
    
    while True:
        user_input = input("\nType your command (or type 'exit' to quit): ").lower()

        if user_input == "exit":
            text_to_speech("Goodbye!")
            break
        
        elif "play" in user_input and "youtube" in user_input:
            play_youtube(user_input.replace("play", "").replace("youtube", "").strip())
        
        else:
            # First search and read a little article
            search_and_read(user_input)
            text_to_speech("Ask me something else or type 'stop' to stop AI mode.")

            # AI conversation loop
            while True:
                follow_up = input("\nType your question (or 'stop' to return to main menu): ").lower()
                if follow_up == "stop":
                    text_to_speech("Okay, returning to command mode.")
                    break
                elif follow_up == "exit":
                    text_to_speech("Goodbye!")
                    return
                else:
                    response = ai_giving_response(follow_up)
                    text_to_speech(response)
#----------------------------------------------------------------------------------------------------
if __name__ == "__main__":
    main()


C:\Users\hankp\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Enter the password to activate assistant:  jarvis

Type your command (or type 'exit' to quit):  play bones on youtube


Opening: https://www.youtube.com/watch?v=ZNsPYmkSPeI



Type your command (or type 'exit' to quit):  tell me 2 sentences about north america


Falling back to Wikipedia due to: https://html.duckduckgo.com/html 202 Ratelimit
Wikipedia Summary:
 Tell Me Why is a 2020 episodic adventure game developed by Dontnod Entertainment and published by Xbox Game Studios. The game consists of three chapters released for Windows and Xbox One in late 2020. It follows a pair of twins, Tyler and Alyson, with the ability to communicate with each other through telepathy, who reunite after ten years in their hometown in rural Alaska and investigate the death of their mother.



Type your question (or 'stop' to return to main menu):  which country does apple operate from?

Type your question (or 'stop' to return to main menu):  stop

Type your command (or type 'exit' to quit):  https://news.sky.com/story/letter-sheds-possible-new-light-on-shakespeares-marriage-13354329 recap this article in 2 sentences


Fetching and summarizing: https://news.sky.com/story/letter-sheds-possible-new-light-on-shakespeares-marriage-13354329



Type your question (or 'stop' to return to main menu):  https://news.sky.com/story/letter-sheds-possible-new-light-on-shakespeares-marriage-13354329 summarise this article in 2 sentences

Type your question (or 'stop' to return to main menu):  stop

Type your command (or type 'exit' to quit):  exit


In [ ]:
tell me the latest news about donald trump